<a href="https://colab.research.google.com/github/KolluruMadhurima/Weather_Forecast_Agent/blob/main/WeatherForecastAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydantic-ai==0.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import nest_asyncio

nest_asyncio.apply()

**WEATHER FORECAST - Function** **Tool**

In [3]:
import requests

BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "6a4a8793a5edcf7e44c42b912dc7671e"

def find_weather(city: str) -> dict:
    """This function returns the current weather forecast for the given city"""
    units = "metric"
    params = {
        'q': city,
        'appid': API_KEY,
        'units': units
    }
    response = requests.get(BASE_URL, params=params)
    result = response.json()
    return result

In [4]:
op = find_weather("pune")
print(op)

{'coord': {'lon': 73.8553, 'lat': 18.5196}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 27.59, 'feels_like': 30.35, 'temp_min': 27.59, 'temp_max': 27.59, 'pressure': 1006, 'humidity': 74, 'sea_level': 1006, 'grnd_level': 934}, 'visibility': 10000, 'wind': {'speed': 4.01, 'deg': 301, 'gust': 5.43}, 'clouds': {'all': 84}, 'dt': 1749705409, 'sys': {'country': 'IN', 'sunrise': 1749688062, 'sunset': 1749735677}, 'timezone': 19800, 'id': 1259229, 'name': 'Pune', 'cod': 200}


***PYDANTIC AGENT***

In [5]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

In [6]:
os.environ["GROQ_API_KEY"] = "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"

In [7]:
class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float

In [8]:
weather_agent = Agent(
    model="groq:llama3-8b-8192",  # ✅ correct format for pydantic_ai
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt="""
You are a helpful weather assistant.
Always use the `get_weather_forecast` tool to answer weather-related questions.
For example:
- "What is the weather in Delhi?" → call tool with city="Delhi"
Only respond after calling the tool with accurate data.
"""
)


In [9]:
# weather forecast tool
@weather_agent.tool
def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
    """
    Returns current weather forecast for a city using OpenWeatherMap API.
    """
    url = "https://api.openweathermap.org/data/2.5/weather"
    api_key = "6a4a8793a5edcf7e44c42b912dc7671e"
    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'
    }

    res = requests.get(url, params=params).json()

    return WeatherForecast(
        location=res["name"],
        description=res["weather"][0]["description"].capitalize(),
        temperature_celsius=res["main"]["temp"]
    )

In [10]:
question = input("🌤️ Ask about the weather: ")
result = weather_agent.run_sync(question)
print("\n📍 Forecast:", result.output)

🌤️ Ask about the weather: delhi

📍 Forecast: The weather in Delhi is Haze with a temperature of 37.05 degrees Celsius.
